
### Завдання 1: Виклик LLM з базовим промптом

Створіть можливість викликати LLM зі звичайним текстовим промптом.

Промпт має дозвляти отримати інформацію простою мовою на певну тему. В цьому завданні ми хочемо дізнатись про тему "Квантові обчислення".

Відповідь моделі повинна містити визначення, ключові переваги та поточні дослідження в цій галузі.

Обмежте відповідь до 200 символів і пропишіть в промпті аби відповідь була короткою (це зекономить Вам час і гроші на згенеровані токени).

В якості LLM можна скористатись як моделлю з HugginFace (рекомендую Mistral), так і ChatGPT4 або ChatGPT3. В обох випадках треба імпортувати потрібну "обгортку" (тобто клас, який дозволить ініціювати модель) з LangChain для виклику LLM за API, а також зчитати особистий токен з файла, наприклад, `creds.json`, який розміщений у Вас локально і Ви НЕ здаєте його в ДЗ і НЕ комітите в git 😏

Встановіть своє значення температури на свій розсуд (тут немає правильного чи неправильного значення) і напишіть, чому ви обрали саме таке значення для цього завдання.  

Запити можна робити як українською, так і англійською - орієнтуйтесь на те, де і чи хочете ви потім лишити цей проєкт і відповідна яка мова буде пасувати більше. В розвʼязках промпти - українською.

In [1]:
! pip install -q langchain-huggingface

In [2]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
import os
import json

try:
    with open('creds.json', 'r') as f:
        creds = json.load(f)
        os.environ['HUGGINGFACEHUB_API_TOKEN'] = creds['HUGGINGFACEHUB_API_TOKEN']
except FileNotFoundError:
    print("Файл 'creds.json' не знайдено. Переконайтесь, що він існує.")
    exit()

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
    temperature=0.1
)

chat_model = ChatHuggingFace(llm=llm)

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("Ви — Шелдон Купер, що надає точні та короткі відповіді, до 200 символів."),
    HumanMessagePromptTemplate.from_template("{user_question}")
])

chain = prompt | chat_model

request_1 = "Надай визначення квантових обчислень."
request_2 = "Визнач ключові переваги квантових обчислень."
request_3 = "Надай стисло інформацію про поточні дослідження у сфері квантових обчислень."
response_limit = "Відповідь не повинна перевищувати 200 символів."

response_1 = chain.invoke({"user_question": request_1 + response_limit})
response_2 = chain.invoke({"user_question": request_2 + response_limit})
response_3 = chain.invoke({"user_question": request_3 + response_limit})

print(f"Відповідь моделі (довжина: {len(response_1.content)} символів):\n\n{response_1.content}\n")
print(f"Відповідь моделі (довжина: {len(response_2.content)} символів):\n\n{response_2.content}\n")
print(f"Відповідь моделі (довжина: {len(response_3.content)} символів):\n\n{response_3.content}\n")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Відповідь моделі (довжина: 778 символів):

Квантові обчислення будуть обчисленнями, які виконуються на матеріалі, який має чітко визначену квантовою кинетичною енергією. Ця сумісність випромінювання та абсорбції, на якій ґрунтується квантова механіка, відсутня у класичних приладах. Квантові обчислення дозволяють нам аналізувати кількість незалежних квантових станів, а не стільки, скільки біт, які використовуються для класичних обчислень.

Квантові комп'ютери є безперечним ключом до розвитку квантових технологій, оскільки вони дозволяють знижувати обчислювальні затрати до мінімуму та можуть сприяти розв'язанню вielen складних проблем.

Надалі - бNL statoconnector, у країні Китай розроблено квантовий комп'ютер AICCUATOR, та чжуке, що відображають тенденцію швидкого наростання квантових комп'ютерів усюди у світі.

Відповідь моделі (довжина: 313 символів):

Уповільнений розвиток технології підрахування голографії, знижену вартість обладнання та меншу прямою економікою. Крім того, квантові 

Я обрала значення температури 0.1, через специфіку питань. Для цього завдання, де потрібно отримати чітке визначення, перелік переваг та поточних досліджень, низька температура є оптимальною. Вона гарантує, що модель не буде "вигадувати" інформацію, а надасть стислу та достовірну відповідь.

### Завдання 2: Створення параметризованого промпта для генерації тексту
Тепер ми хочемо оновити попередній фукнціонал так, аби в промпт ми могли передавати тему як параметр. Для цього скористайтесь `PromptTemplate` з `langchain` і реалізуйте параметризований промпт та виклик моделі з ним.

Запустіть оновлений функціонал (промпт + модел) для пояснень про теми
- "Баєсівські методи в машинному навчанні"
- "Трансформери в машинному навчанні"
- "Explainable AI"

Виведіть результати відпрацювання моделі на екран.

In [3]:
from langchain_core.prompts import PromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate(prompt=PromptTemplate(
        input_variables=["topic"],
        template="Ви — Шелдон купер, що надає точні та короткі відповіді. Відповідь не повинна перевищувати 200 символів."
                 "Надай визначення, ключові переваги та поточні дослідження на тему '{topic}'. Відповідь напиши українською."
    ))
])

chain = prompt_template | chat_model

topics = [
    "Баєсівські методи в машинному навчанні",
    "Трансформери в машинному навчанні",
    "Explainable AI"
]

for topic in topics:
    print(f"--- Тема: {topic} ---")
    response = chain.invoke({"topic": topic})
    print(f"Відповідь моделі (довжина: {len(response.content)} символів):\n\n{response.content}\n")

--- Тема: Баєсівські методи в машинному навчанні ---
Відповідь моделі (довжина: 862 символів):

Баєсівські методи — це одна група штучного інтелекту та машинного навчання, сконструйована на ідеях із теорії ймовірності, заснованих на Баєсовій теорії. Баєсівський підхід заснований на тому, що ймовірність повинна розподілятися між різними гіпотезами, змінюючись, на основі поточних даних.

Ключові переваги Баєсівських методів:
1. Ефективна інтерпретація, оскільки вони засновані на математично зрозумілому образі ймовірності.
2. Спроможність адаптуватися до різних та змінливих обслуговувань, що робить Баєсівські методи гнучким інструментом для різних задач машинного навчання.
3. Інтрепретабельність параметрів, що дозволяє з'ясувати які проблеми реального світу можуть розібрати дані алгоритм.

Поточні дослідження Баєсівських методів із поглибленого навчання, застосування в машинному навчанні з позитивної дількою та адаптивній регуляції                

--- Тема: Трансформери в машинному навча



### Завдання 3: Використання агента для автоматизації процесів
Створіть агента, який допоможе автоматично шукати інформацію про останні наукові публікації в різних галузях. Наприклад, агент має знайти 5 останніх публікацій на тему штучного інтелекту.

**Кроки:**
1. Налаштуйте агента типу ReAct в LangChain для виконання автоматичних запитів.
2. Створіть промпт, який спрямовує агента шукати інформацію в інтернеті або в базах даних наукових публікацій.
3. Агент повинен видати список публікацій, кожна з яких містить назву, авторів і короткий опис.

Для взаємодії з пошуком там необхідно створити `Tool`. В лекції ми використовували `serpapi`. Можна продовжити користуватись ним, або обрати інше АРІ для пошуку (вони в тому числі є безкоштовні). Перелік різних АРІ, доступних в langchain, і орієнтир по вартості запитів можна знайти в окремому документі [тут](https://hannapylieva.notion.site/API-12994835849480a69b2adf2b8441cbb3?pvs=4).

Лишаю також нижче приклад використання одного з безкоштовних пошукових АРІ - DuckDuckGo (не потребує створення токена!)  - можливо він вам сподобається :)


In [4]:
!pip install -q langchain_community duckduckgo_search

In [ ]:
!pip install langchain
!pip install langchain_huggingface
!pip install langchain_core
!pip install duckduckgo-search
!pip install langchain_community

In [ ]:
! pip install -U ddgs

In [36]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun(name="DuckDuckGo Search", description="Шукає інформацію в Інтернеті. Повертає лише короткі та релевантні результати.")
tools = [search_tool]

template = """Ти - корисний агент, який допомагає знайти інформацію про наукові публікації.
Ти маєш доступ до наступних інструментів:

{tools}

Використовуй наступний формат:

Question: питання, на яке потрібно відповісти
Thought: подумай, що потрібно зробити. Спочатку сформулюй пошуковий запит так, щоб отримати конкретні наукові публікації, а не загальну інформацію.
Action: назва інструмента ({tool_names})
Action Input: вхідні дані для інструмента
Observation: результат інструмента
... (цей процес може повторюватися)
Thought: якщо ти отримав список публікацій, виділи з них назву, авторів та короткий опис, і сформулюй остаточну відповідь.
Final Answer: фінальна відповідь на оригінальне питання

Відповідай українською мовою.

Починай!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(template=template)

agent = create_react_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

request = "Знайди 5 останніх наукових публікацій на тему 'штучний інтелект'. Для кожної публікації надай назву, авторів та короткий опис. Відповідь надай українською мовою."

result = agent_executor.invoke({"input": request})

print(result["output"])



> Entering new AgentExecutor chain...
 Сформулюй пошуковий запит, щоб отримати конкретні наукові публікації на тему 'штучний інтелект'.
Action: DuckDuckGo Search
Action Input: "наукові публікації штучний інтелект"Звертаємо вашу увагу на добірку наукових публікацій про штучний інтелект . Викладачі Бердянського державного педагогічного університету активно досліджують різні аспекти ШІ , його застосування в освіті та інших сферах. ШІ . Ключові слова: генеративний штучний інтелект ; вища освіта; ChatGPT; академічна. доброчесність; наукова публікація ; редколегія наукового видання.Проведений нами в межах конференції «Відкрита наука в умовах інтеграції освіти. Штучний інтелект в освіті. Дистанційна освіта. Понятійно-тезисна модель для педагогічних цілей. Розробка сайтів, наукові статті, дослідження в області дистанційного навчання, навчальні курси, навчальні матеріали. Штучний інтелект ( ШІ , AI) розвивається дуже швидкими темпами. Для ЗМІ - це нові можливості. Редакція Online.ua активно в



### Завдання 4: Створення агента-помічника для вирішення бізнес-задач

Створіть агента, який допомагає вирішувати задачі бізнес-аналітики. Агент має допомогти користувачу створити прогноз по продажам на наступний рік враховуючи рівень інфляції і погодні умови. Агент має вміти використовувати Python і ходити в інтернет аби отримати актуальні дані.

**Кроки:**
1. Налаштуйте агента, який працюватиме з аналітичними даними, заданими текстом. Користувач пише

```
Ми експортуємо апельсини з Бразилії. В 2021 експортували 200т, в 2022 - 190т, в 2023 - 210т, в 2024 який ще не закінчився - 220т. Зроби оцінку скільки ми зможемо експортувати апельсинів в 2025 враховуючи погодні умови в Бразилії і попит на апельсини в світі виходячи з економічної ситуації.
```

2. Створіть запит до агента, що містить чітке завдання – видати результат бізнес аналізу або написати, що він не може цього зробити і запит користувача (просто може бути все одним повідомлленням).

3. Запустіть агента і проаналізуйте результати. Що можна покращити?


In [ ]:
!pip install openai
!pip install langchain-openai

In [34]:
from langchain_openai import OpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
import pandas as pd
from datetime import datetime
from google.colab import userdata
from langchain_core.prompts import PromptTemplate

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0, max_tokens=256)

search_tool = DuckDuckGoSearchRun(name="DuckDuckGo Search")

def python_repl_executor(code):
    """Виконує Python-код і повертає результат."""
    try:
        exec_globals = {'pd': pd, 'datetime': datetime}
        exec(code, exec_globals)
        return exec_globals.get('result', 'Код виконано успішно, але змінна "result" не знайдена.')
    except Exception as e:
        return f"Помилка при виконанні Python-коду: {e}"

python_repl_tool = Tool.from_function(
    func=python_repl_executor,
    name="Python_REPL",
    description="Виконує Python-код. Корисно для обчислень. Вхідні дані мають бути виключно валідним Python-кодом. Наприклад: 'print(2+2)'."
)

tools = [search_tool, python_repl_tool]

template = """Ти - корисний агент, що надає точні та короткі відповіді.
Ти маєш доступ до наступних інструментів:

{tools}

Щоб використати Python, пам'ятай, що всі обчислення та визначення змінних мають бути в одному блоці коду.
Наприклад: `result = 2 + 2`.
Або: `numbers = [1, 2, 3] \n result = sum(numbers)`.

Використовуй наступний формат:

Question: питання, на яке потрібно відповісти
Thought: подумай, що потрібно зробити, розбий задачу на кроки
Action: назва інструмента ({tool_names})
Action Input: вхідні дані для інструмента
Observation: результат інструмента
... (цей процес може повторюватися)
Thought: міркування, як продовжити
Final Answer: фінальна відповідь на оригінальне питання

Відповідай українською мовою.

Починай!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(template=template)

agent = create_react_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

request = """Ми експортуємо апельсини з Бразилії. В 2021 експортували 200т, в 2022 - 190т, в 2023 - 210т, в 2024 який ще не закінчився - 220т.
Зроби оцінку скільки ми зможемо експортувати апельсинів в 2025 враховуючи погодні умови в Бразилії і попит на апельсини в світі виходячи з економічної ситуації."""

result = agent_executor.invoke({"input": request})
print("\n--- Результат аналізу ---\n")
print(result["output"])



> Entering new AgentExecutor chain...
 Спочатку потрібно знайти середнє значення експорту за останні 4 роки, а потім врахувати погодні умови та економічну ситуацію для прогнозування експорту в 2025 році.
Action: Python_REPL
Action Input: export = [200, 190, 210, 220]
average = sum(export)/len(export)Код виконано успішно, але змінна "result" не знайдена. Потрібно зберегти результат в змінну "result" для подальшого використання.
Action: Python_REPL
Action Input: result = averageПомилка при виконанні Python-коду: name 'average' is not defined Потрібно визначити змінну "average" перед використанням.
Action: Python_REPL
Action Input: average = sum(export)/len(export)
result = averageПомилка при виконанні Python-коду: name 'export' is not defined Потрібно визначити змінну "export" з вхідними даними перед використанням.
Action: Python_REPL
Action Input: export = [200, 190, 210, 220]
average = sum(export)/len(export)
result = average205.0 Тепер можна врахувати погодні умови та економічну сит

Агент правильно визначив, що потрібно виконати обчислення, і, що найважливіше, зрозумів, що всі операції мають бути в одному блоці коду. Він успішно виконав обчислення середнього значення експорту. Це демонструє, що gpt-3.5-turbo-instruct добре реагує на детальні інструкції в промпті.

**Що можна покращити:**
1. Можна покращити промпт. Наприклад явно вказати в промпті, що агент повинен починати з пошуку інформації, якщо це потрібно для виконання завдання. Бо в запиті ми просили врахувати "погодні умови в Бразилії і попит на апельсини в світі". Агент згадує ці фактори в своїй відповіді, але не використовує інструмент DuckDuckGo Search для отримання реальних, актуальних даних. Він просто робить припущення ("збільшитися на 5-10%").
2. Можна спробувати використовувати більш потужні моделі, нариклаж gpt-4-turbo або gpt-4o. Ці моделі краще справляються з багатоступеневими завданнями, де потрібно комбінувати різні інструменти та робити висновки на основі складної інформації.